# Perspectives on matrix multiplication


Lots of you  seem to have learned how to multiply matrices ([matrix multiplication](https://en.wikipedia.org/wiki/Matrix_multiplication)) in high school.  
We compute the product $C=AB$ of an $m \times n$ matrix $A$ with an $n \times p$ matrix $B$ to produce an $m \times p$ matrix $C$.

Did you ever wonder why "matmul" has such a fancy definition?

When we add matrices we add elements.  Why coudn't matmul be just as easy?

## Compare Elementwise Multiply

Of course the elementwise multiply is doable but never seems to be quite as important:

(I'll bet your high school teacher never mentioned elementwise multiply!)


In [1]:
A=[1 2
   3 4]
B=[7 2
   3 2]
@show(A .* B)    # Elementwise times is the "dot star"
@show(A * B);    # Matmul is just the "star"

A .* B = [7 4; 9 8]
A * B = [13 6; 33 14]


For square n x n matrices, elementwise multiply requires $n^2$ operations, while matmul requires about $2n^3$. (Think $n^2$ dot products, each requiring $n$ mults and almost $n$ adds.)

## Raising the Abstraction

Why is matmul defined this way?  We will find out later in the course when we begin to understand that a matrix represents a linear transformation, and matmul is the natural representation of the composition of transformations.  It is only then you can understand the true nature of matrix multiplication.  (Bet your high school teacher never told you that!)

One of our goals in 18.06 is to sometimes stop thinking of matrices as arrays of numbers, and more as holistic objects.

Abstractly, the rules for matrix multiplication are determined once you define how to multiply matrices by vectors $Ax$, the central [linear operation](https://en.wikipedia.org/wiki/Linear_map) of 18.06, by requiring that multiplication be [associative](https://en.wikipedia.org/wiki/Associative_property).  That is, we require:
$$
A(Bx)=(AB)x
$$
for all matrices $A$ and $B$ and all vectors $x$.  The expression $A(Bx)$ involves only matrix × vector (computing $y=Bx$ then $Ay$), and requiring this to equal $(AB)x$ actually uniquely defines the matrix–matrix product $AB$.

## Perspective 1 (high school!): rows × columns

The  most familar definition is that you take **dot products of rows of A with columns of B** to get the product $C$.  For example:
$$
\begin{pmatrix}
 -14 &   5 & 10 \\
  \color{red}{-5} & -20 & 10 \\
  -6 &  10 &  6
\end{pmatrix} =
\begin{pmatrix}
 2 & -1 & 5 \\
  \color{red}{3} &  \color{red}{4} & \color{red}{4} \\
 -4 & -2 & 0
\end{pmatrix}
\begin{pmatrix}
\color{red}{1}  & 0 & -2 \\
  \color{red}{1} & -5 &  1 \\
 \color{red}{-3} &  0 &  3
\end{pmatrix}
$$
where we have highlighted the entry $\color{red}{-5 = 3 \times 1 + 4 \times 1 + 4 \times -3}$ (second row of $A$ ⋅ first column of $B$).

This can be written out as the formula
$$
c_{ij} = \sum_{k=1}^n a_{ik} b_{kj}
$$
in terms of the entries of the matrices, e.g. $c_{ij}$ is the entry in row $i$, column $j$ of $C$, assuming $A$ has $n$ columns and $B$ has $n$ rows.

Essentially all matrix multiplications in practice are done with a version of this formula — at least, with the same operations, but often the *order* in which you multiply/add individual numbers is re-arranged.

**In this notebook, we will explore several ways to *think* about these operations by re-arranging their order.**

In [2]:
A = [ 2  -1  5
      3   4  4
     -4  -2  0]
B = [ 1   0  -2
      1  -5   1
     -3   0   3]
C = A * B

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

Because matrix multiplication is generally [not commutative](https://en.wikipedia.org/wiki/Commutative_property), $AB$ and $BA$ give *different* matrices:

In [3]:
B*A

3×3 Array{Int64,2}:
  10    3    5
 -17  -23  -15
 -18   -3  -15

In [4]:
A*B - B*A

3×3 Array{Int64,2}:
 -24   2   5
  12   3  25
  12  13  21

Though sometimes it can happen to be commutative.

In [5]:
A*(A^2 + 2*A + inv(A)*10) 

3×3 Array{Float64,2}:
 -138.0  -89.0  -147.0
 -101.0  -24.0    92.0
   44.0   46.0  -132.0

In [6]:
(A^2 + 2*A + inv(A)*10)  * A

3×3 Array{Float64,2}:
 -138.0  -89.0  -147.0
 -101.0  -24.0    92.0
   44.0   46.0  -132.0

If we want, we can compute the individual dot products in Julia too.   For example, let's compute $c_{2,1} = -5$ (the 2nd row and first column of $C$, or `C[2,1]` in Julia) by taking the dot product of the second row of $A$ with the first column of $B$.

To extract rows and columns of a matrix, Julia supports a syntax for "array slicing" pioneered by APL.  The second row of $A$ is `A[2,:]`, and the first column of `B` is `B[:,1]`:

In [7]:
A

3×3 Array{Int64,2}:
  2  -1  5
  3   4  4
 -4  -2  0

In [8]:
A[2,3] # 2nd row, 3rd col

4

In [9]:
A[2,:] # 2nd row of A

3-element Array{Int64,1}:
 3
 4
 4

In [10]:
B

3×3 Array{Int64,2}:
  1   0  -2
  1  -5   1
 -3   0   3

In [11]:
B[:,1] # 1st column of B

3-element Array{Int64,1}:
  1
  1
 -3

Now we can compute $c_{2,1}$ by their dot product via the `dot` function:

In [12]:
dot(A[2,:], B[:,1])

-5

We can also write the dot product with the $x \dot y$ notation:

In [13]:
A[2,:] ⋅ B[:,1]   # type \cdot + tab

-5

In [14]:
α = A[2,3] # \alpha + tab

4

This matches $c_{2,1}$ from above, or `C[2,1]` in Julia:

In [15]:
C[2,1]

-5

In [16]:
A[2,:]' * B[:,1]  # yet another way to get a dot product

-5

### The summation $$c_{ij} = \sum_{k=1}^n a_{ik} b_{kj}$$ directly in a triple loop code

In [17]:
function matmul_ijk(A,B)
   m,n = size(A)
   n2,p = size(B)
   if n≠n2 error("No good, n=$n ≠ n2=$(n2)") end
   
   C = fill(0,m,p) # m x p "zeros" matrix
    
   for i=1:m, j=1:p, k=1:n
      C[i,j] += A[i,k]*B[k,j]   # shorthand for C[i,j] = C[i,j] + A[i,k]*B[k,j] 
   end
    
   return C  
end

matmul_ijk (generic function with 1 method)

In [18]:
matmul_ijk(A,B)

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

## Perspective 2: matrix × columns

$AB$ can be viewed as multiplying $A$ on the *left* by each *column* of $B$.

For example, let's multiply $A$ by the first column of $B$:

In [19]:
A*B

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

In [20]:
B[:,1]

3-element Array{Int64,1}:
  1
  1
 -3

In [21]:
A * B[:,1]

3-element Array{Int64,1}:
 -14
  -5
  -6

In [22]:
using Interact

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2788aaff54f3972bd10a224d42612bcbc5553044-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/2788aaff54f3972bd10a224d42612bcbc5553044-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [23]:
@manipulate for j=1:3
    A * B[:,j]
end

(div
  Widgets.Widget{:slider}(DataStructures.OrderedDict{Symbol,Any}(:changes=>Observables.Observable{Int64}("ob_02", 0, Any[WebIO.SyncCallback(WebIO.Scope("knockout-component-16694083-7eb6-4348-b89a-28d470b67f09", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["j"], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interact "),Pair{Symbol,Any}(:style, Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"text-align:right;width:18%"))), 2), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:max, 3),Pair{Symbol,Any}(:min, 1),Pair{Symbol,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}"))),Pair{Symbol,Any}(:step, 1),Pair{Symbol,Any}(:className, "slider slider is-fullwidth"),Pair{Symbol,Any}(:style, Dict{Any,Any}())), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"flex-grow:1; margin: 0 2%"))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("data-bind"=>"text: displayedvalue"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"width:18%"))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"display:flex; justify-content:center; align-items:center;"))), 7), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("changes", (Observables.Observable{Int64}(#= circular reference @-7 =#), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("value", (Observables.Observable{Int64}("ob_01", 2, Any[WebIO.SyncCallback(WebIO.Scope(#= circular reference @-7 =#), WebIO.SyncCallback(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["knockout"=>"/Users/stevenj/.julia/v0.6/Knockout/src/../assets/knockout.js", "knockout_punches"=>"/Users/stevenj/.julia/v0.6/Knockout/src/../assets/knockout_punches.js", "/Users/stevenj/.julia/v0.6/InteractBase/src/../assets/all.js", "/Users/stevenj/.julia/v0.6/InteractBase/src/../assets/style.css", "/Users/stevenj/.julia/v0.6/InteractBulma/src/../assets/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":2}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        [this[\"displayedvalue\"]=ko.computed(function () {return t

This is the first column of $C$!  If we do this to *all* the columns of $B$, we get $C$:

In [24]:
[ A*B[:,1]  A*B[:,2]  A*B[:,3] ]

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

In [25]:
A*B

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

Equivalently, each column of $B$ specifies a [linear combination](https://en.wikipedia.org/wiki/Linear_combination) of *columns* of $A$ to produce the columns of $C$.   So, **if you want to rearrange the *columns* of a matrix, multiply it by another matrix on the *right***.

For example, let's do the transformation that *flips the sign of the first column of $A$* and *swaps the second and third columns*.

In [26]:
A

3×3 Array{Int64,2}:
  2  -1  5
  3   4  4
 -4  -2  0

In [27]:
A * [ -1  0  0
       0  0  1
       0  1  0  ]

3×3 Array{Int64,2}:
 -2  5  -1
 -3  4   4
  4  0  -2

As another example, let's swap the first two columns:

In [28]:
A * [ 0 1 0
      1 0 0
      0 0 1 ]

3×3 Array{Int64,2}:
 -1   2  5
  4   3  4
 -2  -4  0

In [29]:
A

3×3 Array{Int64,2}:
  2  -1  5
  3   4  4
 -4  -2  0

A lot of students are perplexed.  They wonder how it could be legal to reorder in this way. 
It might take working through a few examples by hand to realize that from the perspective
of C[i,j], the same sum is accumulated in the same order, but the order in which the different elements of C finish may vary. This little Julia demo may help with this understanding.

## Perspective 3: rows × matrix

$AB$ can be viewed as multiplying each *row* of $A$ by the matrix $B$ on the *right*.  Multiplying a [row vector](https://en.wikipedia.org/wiki/Row_and_column_vectors) by a matrix on the right produces another row vector.

For example, here is the first row of $A$:

In [30]:
A[1,:]

3-element Array{Int64,1}:
  2
 -1
  5

Whoops, slicing a matrix in Julia produces a 1d array, which is interpreted as a column vector, no matter how you slice it.  We can't multiply a column vector by a matrix $B$ on the *right* — that operation is not defined in linear algebra (the dimensions don't match up).  Julia will give an error if we try it:

In [31]:
A

3×3 Array{Int64,2}:
  2  -1  5
  3   4  4
 -4  -2  0

In [32]:
A[1,:] * B

LoadError: [91mDimensionMismatch("matrix A has dimensions (3,1), matrix B has dimensions (3,3)")[39m

To get a row vector we must [transpose](https://en.wikipedia.org/wiki/Transpose) the slice `A[1,:]`.  In linear algebra, the transpose of a vector $x$ is usually denoted $x^T$.   In Julia, the transpose is `x.'`.

If we omit the `.` and just write `x'` it is the [complex-conjugate of the transpose](https://en.wikipedia.org/wiki/Conjugate_transpose), sometimes called the *adjoint*, often denoted $x^H$ (in matrix textbooks), $x^*$ (in pure math), or $x^\dagger$ (in physics).  For real-valued vectors (no complex numbers), the conjugate transpose is the same as the transpose, and correspondingly we usually just do `x'` for real vectors.

In [33]:
A[1,:]'

1×3 RowVector{Int64,Array{Int64,1}}:
 2  -1  5

Now, let's multiply this by $B$, which should give the first *row* of $C$:

In [34]:
A[1,:]' * B

1×3 RowVector{Int64,Array{Int64,1}}:
 -14  5  10

In [35]:
C

3×3 Array{Int64,2}:
 -14    5  10
  -5  -20  10
  -6   10   6

Yup!

Note that if we multiply a row vector by a matrix on the *left*, it doesn't really make sense.  Julia will give an error:

In [36]:
B * A[1,:]'

LoadError: [91mDimensionMismatch("matrix A has dimensions (3,3), matrix B has dimensions (1,3)")[39m

If we multiply $B$ on the right by *all* the rows of $A$, we get $C$ again:

In [37]:
[ A[1,:]'*B 
  A[2,:]'*B
  A[3,:]'*B ] == C

true

Equivalently, each row of $A$ specifies a linear combination of *rows* of $B$ to produce the rows of $C$.   So, **if you want to rearrange the *rows* of a matrix, multiply it by another matrix on the *left***.

For example, let's do the transformation that *adds two times the first row of $B$ to the third row, and leaves the other rows untouched*.  This is resembles one of the steps of Gaussian elimination!

In [38]:
B

3×3 Array{Int64,2}:
  1   0  -2
  1  -5   1
 -3   0   3

In [39]:
[ 1 0 0
  0 1 0
  2 0 1 ] * B

3×3 Array{Int64,2}:
  1   0  -2
  1  -5   1
 -1   0  -1

In fact, we can write *exactly* one of the steps for Gaussian elimination on $B$ in this form:

Let's do the transformation that *adds three times the first row of $B$ to the third row, subtracts the first row from the second, and leaves the first row untouched*.  

In [40]:
[ 1 0 0
 -1 1 0
  3 0 1 ] * B

3×3 Array{Int64,2}:
 1   0  -2
 0  -5   3
 0   0  -3

As desired, this "eliminates" the sub-diagonal entries of the first column of B.   The nice thing about writing elimination algebraically in this way, instead of in words, is that it will allow us to *think* about the elimination process algebraically.

## Perspective 4: columns × rows

The key to this perspective is to observe:

* elements in column $i$ of $A$ only multiply elements in row $j$ of $B$
* a column times a row vector, sometimes denoted $xy^T$, is an [outer product](https://en.wikipedia.org/wiki/Outer_product) and produces a "rank-1" *matrix*

(See [this excellent paper by Gil Strang](https://doi.org/10.1080/00029890.2018.1408378) for more on this perspective applied to linear algebra. You will be in a better position to understand this at the end of 18.06, however.)

For example, here is column 1 of $A$ times row 1 of $B$:

In [41]:
A[:,1] * B[1,:]'

3×3 Array{Int64,2}:
  2  0  -4
  3  0  -6
 -4  0   8

If we do this for all three rows and columns and add them up, we get $C$:

In [42]:
A[:,1] * B[1,:]' + A[:,2] * B[2,:]' + A[:,3] * B[3,:]' == C

true

So, from this perspective, we could write:

$$
AB = \sum_{k=1}^3 (\mbox{column } k \mbox{ of } A) (\mbox{row } k \mbox{ of } B) = \sum_{k=1}^3 A[:,k] \, B[k,:]^T
$$

where in the last expression we have used Julia notation for slices.

## Perspective 5: submatrix blocks × blocks

It turns out that all of the above are special cases of a more general rule, by which we can break up a matrix in to "submatrix" blocks and multiply the blocks.  Rows, columns, etc. are just blocks of different shapes.  We will do more of this later in the course.